In [1]:
# ╔═══════════════════════════════════════════════════════════════════════════╗
# ║                                                                           ║
# ║   SHA-256 COLLISION FINDER v2.0 - EXTENDED                               ║
# ║   N-Dimensional Space + Non-Linear Transformations                        ║
# ║   + Iterative Attractors + Reduced-Round Analysis                         ║
# ║                                                                           ║
# ║   Author: Kaoru Aguilera Katayama                                        ║
# ║   Extended Version                                                        ║
# ║                                                                           ║
# ╚═══════════════════════════════════════════════════════════════════════════╝

import hashlib
import numpy as np
from collections import defaultdict
from typing import Dict, List, Tuple, Optional
import time
import struct
import warnings
warnings.filterwarnings('ignore')

# ═══════════════════════════════════════════════════════════════════════════
# CONSTANTS
# ═══════════════════════════════════════════════════════════════════════════

# SHA-256 Constants (primeros 32 bits de las raíces cúbicas de los primeros 64 primos)
SHA256_K = [
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
]

# Initial hash values
SHA256_H = [
    0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
    0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19
]

# ═══════════════════════════════════════════════════════════════════════════
# BANNER
# ═══════════════════════════════════════════════════════════════════════════

def show_banner():
    print("""
╔═══════════════════════════════════════════════════════════════════════════╗
║                                                                           ║
║   ███████╗██╗  ██╗ █████╗       ██████╗ ███████╗ ██████╗                  ║
║   ██╔════╝██║  ██║██╔══██╗      ╚════██╗██╔════╝██╔════╝                  ║
║   ███████╗███████║███████║█████╗ █████╔╝███████╗███████╗                  ║
║   ╚════██║██╔══██║██╔══██║╚════╝██╔═══╝ ╚════██║██╔═══██╗                 ║
║   ███████║██║  ██║██║  ██║      ███████╗███████║╚██████╔╝                 ║
║   ╚══════╝╚═╝  ╚═╝╚═╝  ╚═╝      ╚══════╝╚══════╝ ╚═════╝                  ║
║                                                                           ║
║              COLLISION FINDER v2.0 - EXTENDED                             ║
║                                                                           ║
║   🔷 N-Dimensional Embeddings (4D, 5D, ... nD)                           ║
║   🔷 Non-Linear Dimensional Transformations                               ║
║   🔷 Iterative Attractor Refinement                                       ║
║   🔷 Reduced-Round SHA-256 Analysis                                       ║
║                                                                           ║
║   Author: Kaoru Aguilera Katayama                                        ║
║                                                                           ║
╚═══════════════════════════════════════════════════════════════════════════╝
    """)

def show_theory_extended():
    print("""
═══════════════════════════════════════════════════════════════════════════
              N-DIMENSIONAL SPACE THEORY - EXTENDED
═══════════════════════════════════════════════════════════════════════════

🔷 HIGHER DIMENSIONAL EMBEDDINGS:

   3D Space (Original):
      X = SHA(m) normalized
      Y = message index
      Z = collision indicator

   4D Space (Extended):
      X₁ = SHA bits [0:64] normalized
      X₂ = SHA bits [64:128] normalized
      X₃ = SHA bits [128:192] normalized
      X₄ = SHA bits [192:256] normalized

   nD Space (Generalized):
      Split 256 bits into n chunks of 256/n bits each

🔷 NON-LINEAR TRANSFORMATIONS:

   • Polynomial: Y = aX³ + bX² + cX + d
   • Exponential decay mapping
   • Trigonometric phase correlation
   • Wavelet decomposition
   • Manifold learning (local geometry)

🔷 ATTRACTOR THEORY:

   Hash collisions = Strange attractors in hash space
   Iteration: X_{n+1} = f(X_n) until convergence

🔷 REDUCED-ROUND SHA-256:

   Full SHA-256: 64 rounds (computationally secure)
   Reduced versions for analysis:
   • 16 rounds - Very weak
   • 24 rounds - Weak
   • 32 rounds - Moderate
   • 48 rounds - Strong

═══════════════════════════════════════════════════════════════════════════
    """)

# ═══════════════════════════════════════════════════════════════════════════
# CORE UTILITIES
# ═══════════════════════════════════════════════════════════════════════════

def sha256_int(msg) -> int:
    """SHA-256 as 256-bit integer"""
    return int(hashlib.sha256(str(msg).encode()).hexdigest(), 16)

def sha256_hex(msg) -> str:
    """SHA-256 as hex string"""
    return hashlib.sha256(str(msg).encode()).hexdigest()

def sha256_bits(msg) -> str:
    """SHA-256 as binary string"""
    return bin(sha256_int(msg))[2:].zfill(256)

def count_equal_bits(h1: str, h2: str) -> int:
    """Count identical bits between two hashes"""
    bin1 = bin(int(h1, 16))[2:].zfill(256)
    bin2 = bin(int(h2, 16))[2:].zfill(256)
    return sum(1 for a, b in zip(bin1, bin2) if a == b)

def hamming_distance(h1: str, h2: str) -> int:
    """Hamming distance between two hashes"""
    return 256 - count_equal_bits(h1, h2)

def hash_to_vector(h: str, dimensions: int = 8) -> np.ndarray:
    """Convert hash to n-dimensional vector"""
    bits_per_dim = 256 // dimensions
    h_int = int(h, 16)
    vector = []

    for i in range(dimensions):
        chunk = (h_int >> (i * bits_per_dim)) & ((1 << bits_per_dim) - 1)
        normalized = chunk / (2 ** bits_per_dim)
        vector.append(normalized)

    return np.array(vector)

# ═══════════════════════════════════════════════════════════════════════════
# REDUCED-ROUND SHA-256 IMPLEMENTATION
# ═══════════════════════════════════════════════════════════════════════════

class ReducedSHA256:
    """SHA-256 with configurable number of rounds (for analysis)"""

    def __init__(self, rounds: int = 64):
        self.rounds = min(rounds, 64)
        self.k = SHA256_K[:self.rounds]

    def _rotr(self, x: int, n: int) -> int:
        return ((x >> n) | (x << (32 - n))) & 0xffffffff

    def _ch(self, x: int, y: int, z: int) -> int:
        return (x & y) ^ (~x & z)

    def _maj(self, x: int, y: int, z: int) -> int:
        return (x & y) ^ (x & z) ^ (y & z)

    def _sigma0(self, x: int) -> int:
        return self._rotr(x, 2) ^ self._rotr(x, 13) ^ self._rotr(x, 22)

    def _sigma1(self, x: int) -> int:
        return self._rotr(x, 6) ^ self._rotr(x, 11) ^ self._rotr(x, 25)

    def _gamma0(self, x: int) -> int:
        return self._rotr(x, 7) ^ self._rotr(x, 18) ^ (x >> 3)

    def _gamma1(self, x: int) -> int:
        return self._rotr(x, 17) ^ self._rotr(x, 19) ^ (x >> 10)

    def hash(self, message: bytes) -> str:
        """Compute reduced-round SHA-256"""

        # Padding
        msg_len = len(message)
        message += b'\x80'
        message += b'\x00' * ((55 - msg_len) % 64)
        message += struct.pack('>Q', msg_len * 8)

        # Initialize hash values
        h = list(SHA256_H)

        # Process each 512-bit block
        for i in range(0, len(message), 64):
            block = message[i:i+64]

            # Create message schedule
            w = list(struct.unpack('>16I', block))
            for j in range(16, 64):
                w.append((self._gamma1(w[j-2]) + w[j-7] +
                         self._gamma0(w[j-15]) + w[j-16]) & 0xffffffff)

            # Initialize working variables
            a, b, c, d, e, f, g, hh = h

            # Main loop (reduced rounds)
            for j in range(self.rounds):
                t1 = (hh + self._sigma1(e) + self._ch(e, f, g) +
                      self.k[j] + w[j]) & 0xffffffff
                t2 = (self._sigma0(a) + self._maj(a, b, c)) & 0xffffffff

                hh = g
                g = f
                f = e
                e = (d + t1) & 0xffffffff
                d = c
                c = b
                b = a
                a = (t1 + t2) & 0xffffffff

            # Update hash values
            h[0] = (h[0] + a) & 0xffffffff
            h[1] = (h[1] + b) & 0xffffffff
            h[2] = (h[2] + c) & 0xffffffff
            h[3] = (h[3] + d) & 0xffffffff
            h[4] = (h[4] + e) & 0xffffffff
            h[5] = (h[5] + f) & 0xffffffff
            h[6] = (h[6] + g) & 0xffffffff
            h[7] = (h[7] + hh) & 0xffffffff

        return ''.join(f'{x:08x}' for x in h)

    def hash_str(self, msg: str) -> str:
        return self.hash(str(msg).encode())

# ═══════════════════════════════════════════════════════════════════════════
# N-DIMENSIONAL EMBEDDING
# ═══════════════════════════════════════════════════════════════════════════

class NDimensionalSpace:
    """N-Dimensional hash space embedding"""

    def __init__(self, dimensions: int = 4):
        self.dimensions = dimensions
        self.points = []
        self.hashes = []
        self.messages = []

    def add_point(self, message: int, hash_hex: str):
        """Add a point to the space"""
        vector = hash_to_vector(hash_hex, self.dimensions)
        self.points.append(vector)
        self.hashes.append(hash_hex)
        self.messages.append(message)

    def build_space(self, N: int, hash_func=sha256_hex, show_progress: bool = True):
        """Build N-dimensional space with N points"""

        if show_progress:
            print(f"\n   📐 Building {self.dimensions}D space with {N:,} points...")

        for i in range(1, N + 1):
            h = hash_func(i)
            self.add_point(i, h)

            if show_progress and i % 25000 == 0:
                print(f"      Processed: {i:,}/{N:,}")

        self.points = np.array(self.points)

        if show_progress:
            print(f"   ✓ {self.dimensions}D space built: shape = {self.points.shape}")

        return self

    def find_nearest_neighbors(self, k: int = 10) -> List[Tuple[int, int, float]]:
        """Find k-nearest neighbors for collision candidates"""

        print(f"\n   🔍 Finding nearest neighbors in {self.dimensions}D space...")

        from scipy.spatial import KDTree

        tree = KDTree(self.points)

        candidates = []

        for i, point in enumerate(self.points):
            distances, indices = tree.query(point, k=k+1)

            for j, (dist, idx) in enumerate(zip(distances[1:], indices[1:])):
                if idx != i:
                    candidates.append((self.messages[i], self.messages[idx], dist))

        # Sort by distance
        candidates.sort(key=lambda x: x[2])

        # Remove duplicates
        seen = set()
        unique = []
        for m1, m2, dist in candidates:
            key = tuple(sorted([m1, m2]))
            if key not in seen:
                seen.add(key)
                unique.append((m1, m2, dist))

        return unique[:1000]  # Top 1000 candidates

    def calculate_centroid(self) -> np.ndarray:
        """Calculate centroid of all points"""
        return np.mean(self.points, axis=0)

    def find_cluster_centers(self, n_clusters: int = 10) -> np.ndarray:
        """Find cluster centers using K-means"""

        print(f"\n   📊 Finding {n_clusters} cluster centers...")

        # Simple K-means
        centers = self.points[np.random.choice(len(self.points), n_clusters, replace=False)]

        for iteration in range(50):
            # Assign points to nearest center
            distances = np.linalg.norm(self.points[:, np.newaxis] - centers, axis=2)
            assignments = np.argmin(distances, axis=1)

            # Update centers
            new_centers = np.array([
                self.points[assignments == k].mean(axis=0) if np.sum(assignments == k) > 0 else centers[k]
                for k in range(n_clusters)
            ])

            if np.allclose(centers, new_centers):
                break
            centers = new_centers

        return centers

# ═══════════════════════════════════════════════════════════════════════════
# NON-LINEAR TRANSFORMATIONS
# ═══════════════════════════════════════════════════════════════════════════

class NonLinearTransformer:
    """Non-linear dimensional transformations"""

    @staticmethod
    def polynomial_transform(points: np.ndarray, degree: int = 3) -> np.ndarray:
        """Polynomial transformation: expand to higher degree terms"""

        print(f"   🔄 Applying polynomial transform (degree={degree})...")

        n_samples, n_features = points.shape

        # Generate polynomial features
        transformed = [points]

        for d in range(2, degree + 1):
            transformed.append(points ** d)

        # Cross terms (for degree 2)
        if degree >= 2:
            for i in range(n_features):
                for j in range(i + 1, n_features):
                    transformed.append((points[:, i] * points[:, j]).reshape(-1, 1))

        return np.hstack(transformed)

    @staticmethod
    def exponential_decay_transform(points: np.ndarray, alpha: float = 2.0) -> np.ndarray:
        """Exponential decay mapping"""

        print(f"   🔄 Applying exponential decay transform (α={alpha})...")

        return np.exp(-alpha * points)

    @staticmethod
    def trigonometric_transform(points: np.ndarray, frequencies: List[float] = [1, 2, 4]) -> np.ndarray:
        """Trigonometric phase correlation"""

        print(f"   🔄 Applying trigonometric transform...")

        transformed = []

        for freq in frequencies:
            transformed.append(np.sin(2 * np.pi * freq * points))
            transformed.append(np.cos(2 * np.pi * freq * points))

        return np.hstack(transformed)

    @staticmethod
    def wavelet_transform(points: np.ndarray, scales: List[float] = [1, 2, 4, 8]) -> np.ndarray:
        """Simple wavelet-like decomposition using Ricker wavelet"""

        print(f"   🔄 Applying wavelet transform...")

        def ricker(t, sigma):
            """Ricker (Mexican hat) wavelet"""
            return (1 - (t/sigma)**2) * np.exp(-(t/sigma)**2 / 2)

        transformed = []

        for scale in scales:
            for i in range(points.shape[1]):
                # Center the data
                centered = points[:, i] - np.mean(points[:, i])
                # Apply wavelet
                wavelet_coeff = ricker(centered, scale)
                transformed.append(wavelet_coeff.reshape(-1, 1))

        return np.hstack(transformed)

    @staticmethod
    def manifold_projection(points: np.ndarray, target_dim: int = 3) -> np.ndarray:
        """Project onto lower-dimensional manifold using local geometry"""

        print(f"   🔄 Applying manifold projection to {target_dim}D...")

        # Center the data
        centered = points - np.mean(points, axis=0)

        # Compute covariance matrix
        cov = np.cov(centered.T)

        # Eigendecomposition
        eigenvalues, eigenvectors = np.linalg.eigh(cov)

        # Sort by eigenvalue (descending)
        idx = np.argsort(eigenvalues)[::-1]
        eigenvectors = eigenvectors[:, idx]

        # Project onto top components
        projection = centered @ eigenvectors[:, :target_dim]

        return projection

    @staticmethod
    def radial_basis_transform(points: np.ndarray, n_centers: int = 50) -> np.ndarray:
        """Radial Basis Function transformation"""

        print(f"   🔄 Applying RBF transform ({n_centers} centers)...")

        # Select random centers
        center_idx = np.random.choice(len(points), n_centers, replace=False)
        centers = points[center_idx]

        # Compute gamma
        gamma = 1.0 / (2 * np.var(points))

        # Compute RBF features
        transformed = np.zeros((len(points), n_centers))

        for i, center in enumerate(centers):
            distances = np.linalg.norm(points - center, axis=1)
            transformed[:, i] = np.exp(-gamma * distances ** 2)

        return transformed

# ═══════════════════════════════════════════════════════════════════════════
# ITERATIVE ATTRACTOR REFINEMENT
# ═══════════════════════════════════════════════════════════════════════════

class AttractorRefinement:
    """Iterative attractor-based collision search"""

    def __init__(self, hash_func=sha256_hex):
        self.hash_func = hash_func
        self.attractors = []
        self.best_collision = None

    def initialize_attractors(self, seeds: List[int], radius: int = 1000):
        """Initialize attractors from seed points"""

        print(f"\n   🎯 Initializing {len(seeds)} attractors...")

        for seed in seeds:
            attractor = {
                'center': seed,
                'radius': radius,
                'best_pair': None,
                'best_bits': 0,
                'history': []
            }
            self.attractors.append(attractor)

    def evaluate_neighborhood(self, center: int, radius: int, samples: int = 1000) -> Dict:
        """Evaluate collision potential in neighborhood"""

        hashes = {}

        # Sample points in neighborhood
        sample_points = [center]
        for _ in range(samples - 1):
            offset = np.random.randint(-radius, radius + 1)
            point = max(1, center + offset)
            sample_points.append(point)

        # Compute hashes
        for point in sample_points:
            hashes[point] = self.hash_func(point)

        # Find best pair in neighborhood
        best_pair = None
        best_bits = 0

        points = list(hashes.keys())
        for i in range(len(points)):
            for j in range(i + 1, len(points)):
                p1, p2 = points[i], points[j]
                bits = count_equal_bits(hashes[p1], hashes[p2])

                if bits > best_bits:
                    best_bits = bits
                    best_pair = (p1, p2, hashes[p1], hashes[p2])

        return {
            'center': center,
            'best_pair': best_pair,
            'best_bits': best_bits,
            'gradient': self._estimate_gradient(center, hashes)
        }

    def _estimate_gradient(self, center: int, hashes: Dict) -> int:
        """Estimate direction of improvement"""

        # Check neighbors
        left = max(1, center - 100)
        right = center + 100

        h_center = hashes.get(center, self.hash_func(center))
        h_left = hashes.get(left, self.hash_func(left))
        h_right = hashes.get(right, self.hash_func(right))

        # Compare with all other hashes
        best_left = max(count_equal_bits(h_left, h) for h in hashes.values() if h != h_left)
        best_right = max(count_equal_bits(h_right, h) for h in hashes.values() if h != h_right)

        if best_left > best_right:
            return -1  # Move left
        elif best_right > best_left:
            return 1   # Move right
        else:
            return 0   # Stay

    def iterate(self, iterations: int = 10, samples_per_iter: int = 500):
        """Main iteration loop"""

        print(f"\n   🔄 Running {iterations} attractor iterations...")

        global_best_bits = 0
        global_best_pair = None

        for iter_num in range(iterations):
            print(f"\n   Iteration {iter_num + 1}/{iterations}")

            for i, attractor in enumerate(self.attractors):
                # Evaluate neighborhood
                result = self.evaluate_neighborhood(
                    attractor['center'],
                    attractor['radius'],
                    samples_per_iter
                )

                # Update attractor
                if result['best_bits'] > attractor['best_bits']:
                    attractor['best_bits'] = result['best_bits']
                    attractor['best_pair'] = result['best_pair']

                    if result['best_bits'] > global_best_bits:
                        global_best_bits = result['best_bits']
                        global_best_pair = result['best_pair']
                        print(f"      ✨ Attractor {i}: NEW BEST = {result['best_bits']}/256 bits")

                # Move attractor based on gradient
                gradient = result['gradient']
                step = int(attractor['radius'] * 0.3 * gradient)
                attractor['center'] = max(1, attractor['center'] + step)

                # Shrink radius over time
                attractor['radius'] = max(50, int(attractor['radius'] * 0.9))

                # Record history
                attractor['history'].append({
                    'center': attractor['center'],
                    'best_bits': attractor['best_bits']
                })

        self.best_collision = global_best_pair
        return global_best_pair, global_best_bits

    def refine_best(self, pair: Tuple, max_iterations: int = 100):
        """Fine-grained refinement of best pair"""

        if pair is None:
            return None

        print(f"\n   🔬 Fine-grained refinement...")

        m1, m2 = pair[0], pair[1]
        best_bits = count_equal_bits(self.hash_func(m1), self.hash_func(m2))
        best_pair = (m1, m2)

        for iteration in range(max_iterations):
            improved = False

            # Try small perturbations
            for delta1 in range(-10, 11):
                for delta2 in range(-10, 11):
                    new_m1 = max(1, m1 + delta1)
                    new_m2 = max(1, m2 + delta2)

                    if new_m1 != new_m2:
                        h1 = self.hash_func(new_m1)
                        h2 = self.hash_func(new_m2)
                        bits = count_equal_bits(h1, h2)

                        if bits > best_bits:
                            best_bits = bits
                            best_pair = (new_m1, new_m2)
                            m1, m2 = new_m1, new_m2
                            improved = True
                            print(f"      Iteration {iteration}: {bits}/256 bits")

            if not improved:
                break

        h1 = self.hash_func(best_pair[0])
        h2 = self.hash_func(best_pair[1])

        return (best_pair[0], best_pair[1], h1, h2), best_bits

# ═══════════════════════════════════════════════════════════════════════════
# HYBRID COLLISION FINDER
# ═══════════════════════════════════════════════════════════════════════════

class HybridCollisionFinder:
    """Combines all methods for collision finding"""

    def __init__(self, dimensions: int = 8, rounds: int = 64):
        self.dimensions = dimensions
        self.rounds = rounds
        self.nd_space = NDimensionalSpace(dimensions)
        self.transformer = NonLinearTransformer()

        if rounds < 64:
            self.sha = ReducedSHA256(rounds)
            self.hash_func = self.sha.hash_str
            print(f"   ⚠️  Using REDUCED SHA-256 ({rounds} rounds)")
        else:
            self.hash_func = sha256_hex
            print(f"   🔒 Using FULL SHA-256 (64 rounds)")

        self.best_result = {
            'bits': 0,
            'm1': None,
            'm2': None,
            'h1': None,
            'h2': None,
            'method': None
        }

    def update_best(self, bits: int, m1: int, m2: int, h1: str, h2: str, method: str):
        """Update best result if improved"""
        if bits > self.best_result['bits']:
            self.best_result = {
                'bits': bits,
                'm1': m1,
                'm2': m2,
                'h1': h1,
                'h2': h2,
                'method': method
            }
            return True
        return False

    def run_nd_analysis(self, N: int = 50000):
        """Run N-dimensional analysis"""

        print("\n" + "="*70)
        print(f"📐 N-DIMENSIONAL ANALYSIS ({self.dimensions}D)")
        print("="*70)

        # Build space
        self.nd_space.build_space(N, self.hash_func)

        # Find nearest neighbors
        try:
            candidates = self.nd_space.find_nearest_neighbors(k=20)

            print(f"\n   Evaluating {len(candidates)} nearest neighbor candidates...")

            for m1, m2, dist in candidates[:500]:
                h1 = self.hash_func(m1)
                h2 = self.hash_func(m2)
                bits = count_equal_bits(h1, h2)

                if self.update_best(bits, m1, m2, h1, h2, f"{self.dimensions}D-NN"):
                    print(f"      ✨ NEW: {bits}/256 bits - M1={m1}, M2={m2}")

        except ImportError:
            print("   ⚠️  scipy not available, using fallback method...")
            self._fallback_nn_search(N)

    def _fallback_nn_search(self, N: int):
        """Fallback nearest neighbor search without scipy"""

        print("   Using brute-force neighbor search...")

        # Sample random pairs
        for _ in range(10000):
            i = np.random.randint(1, N + 1)
            j = np.random.randint(1, N + 1)

            if i != j:
                h1 = self.hash_func(i)
                h2 = self.hash_func(j)
                bits = count_equal_bits(h1, h2)

                if self.update_best(bits, i, j, h1, h2, f"{self.dimensions}D-Random"):
                    print(f"      ✨ NEW: {bits}/256 bits - M1={i}, M2={j}")

    def run_nonlinear_analysis(self):
        """Run non-linear transformation analysis"""

        print("\n" + "="*70)
        print("🔄 NON-LINEAR TRANSFORMATION ANALYSIS")
        print("="*70)

        if len(self.nd_space.points) == 0:
            print("   ⚠️  Run N-dimensional analysis first!")
            return

        points = self.nd_space.points

        # Apply transformations
        transformations = [
            ("Polynomial", self.transformer.polynomial_transform(points, degree=2)),
            ("Exponential", self.transformer.exponential_decay_transform(points)),
            ("Trigonometric", self.transformer.trigonometric_transform(points)),
            ("Manifold", self.transformer.manifold_projection(points, target_dim=3)),
        ]

        for name, transformed in transformations:
            print(f"\n   Analyzing {name} transform (shape: {transformed.shape})...")

            # Find clusters in transformed space
            try:
                from scipy.spatial import KDTree
                tree = KDTree(transformed)

                # Sample points and find neighbors
                for i in range(0, len(transformed), max(1, len(transformed) // 100)):
                    _, indices = tree.query(transformed[i], k=10)

                    for idx in indices[1:]:
                        if idx != i and idx < len(self.nd_space.messages):
                            m1 = self.nd_space.messages[i]
                            m2 = self.nd_space.messages[idx]
                            h1 = self.nd_space.hashes[i]
                            h2 = self.nd_space.hashes[idx]
                            bits = count_equal_bits(h1, h2)

                            if self.update_best(bits, m1, m2, h1, h2, f"NL-{name}"):
                                print(f"      ✨ {name}: {bits}/256 bits")

            except ImportError:
                # Fallback without scipy
                pass

    def run_attractor_analysis(self, n_attractors: int = 20, iterations: int = 10):
        """Run attractor-based analysis"""

        print("\n" + "="*70)
        print("🎯 ATTRACTOR REFINEMENT ANALYSIS")
        print("="*70)

        attractor = AttractorRefinement(self.hash_func)

        # Initialize attractors at promising points
        if self.best_result['m1'] is not None:
            seeds = [self.best_result['m1'], self.best_result['m2']]
        else:
            seeds = []

        # Add random seeds
        while len(seeds) < n_attractors:
            seeds.append(np.random.randint(1, 100000))

        attractor.initialize_attractors(seeds[:n_attractors])

        # Run iterations
        result, bits = attractor.iterate(iterations=iterations)

        if result is not None:
            m1, m2, h1, h2 = result
            if self.update_best(bits, m1, m2, h1, h2, "Attractor"):
                print(f"\n   ✨ Attractor result: {bits}/256 bits")

        # Fine-grained refinement
        if result is not None:
            refined, refined_bits = attractor.refine_best(result)
            if refined is not None:
                m1, m2, h1, h2 = refined
                if self.update_best(refined_bits, m1, m2, h1, h2, "Attractor-Refined"):
                    print(f"   ✨ Refined: {refined_bits}/256 bits")

    def run_prefix_attack(self, N: int = 100000):
        """Run prefix-based collision search"""

        print("\n" + "="*70)
        print("🔍 PREFIX GROUPING ATTACK")
        print("="*70)

        print(f"\n   Building prefix table for {N:,} messages...")

        prefixes = defaultdict(list)

        for i in range(1, N + 1):
            h = self.hash_func(i)

            for length in [4, 5, 6, 7, 8]:
                prefixes[(length, h[:length])].append((i, h))

            if i % 25000 == 0:
                print(f"      Processed: {i:,}/{N:,}")

        # Check groups
        for length in [8, 7, 6, 5, 4]:
            groups = [(p, g) for p, g in prefixes.items() if p[0] == length and len(g) >= 2]

            print(f"\n   Checking {len(groups)} groups with {length}-char prefix match...")

            for (_, prefix), group in groups:
                for i in range(len(group)):
                    for j in range(i + 1, len(group)):
                        m1, h1 = group[i]
                        m2, h2 = group[j]

                        if m1 != m2:
                            bits = count_equal_bits(h1, h2)

                            if self.update_best(bits, m1, m2, h1, h2, f"Prefix-{length}"):
                                print(f"      ✨ Prefix-{length}: {bits}/256 bits - M1={m1}, M2={m2}")

    def run_differential_analysis(self, base_messages: List[int] = None, variations: int = 1000):
        """Differential cryptanalysis approach"""

        print("\n" + "="*70)
        print("⚡ DIFFERENTIAL ANALYSIS")
        print("="*70)

        if base_messages is None:
            base_messages = list(range(1, 101))

        print(f"\n   Analyzing {len(base_messages)} base messages with {variations} variations each...")

        for base in base_messages:
            h_base = self.hash_func(base)

            # Try systematic bit flips in the message
            for v in range(1, variations + 1):
                # XOR-based variation
                variant = base ^ v
                if variant > 0 and variant != base:
                    h_var = self.hash_func(variant)
                    bits = count_equal_bits(h_base, h_var)

                    if self.update_best(bits, base, variant, h_base, h_var, "Differential-XOR"):
                        print(f"      ✨ XOR diff: {bits}/256 bits - {base} ^ {v} = {variant}")

                # Additive variation
                variant_add = base + v
                h_add = self.hash_func(variant_add)
                bits = count_equal_bits(h_base, h_add)

                if self.update_best(bits, base, variant_add, h_base, h_add, "Differential-Add"):
                    print(f"      ✨ Add diff: {bits}/256 bits - {base} + {v}")

    def run_full_analysis(self, N: int = 50000, iterations: int = 10):
        """Run complete analysis pipeline"""

        print("\n" + "="*70)
        print("🚀 RUNNING FULL ANALYSIS PIPELINE")
        print("="*70)

        start_time = time.time()

        # Phase 1: N-dimensional
        self.run_nd_analysis(N)

        # Phase 2: Non-linear transforms
        self.run_nonlinear_analysis()

        # Phase 3: Attractors
        self.run_attractor_analysis(n_attractors=20, iterations=iterations)

        # Phase 4: Prefix attack
        self.run_prefix_attack(N)

        # Phase 5: Differential
        self.run_differential_analysis()

        elapsed = time.time() - start_time

        return elapsed

# ═══════════════════════════════════════════════════════════════════════════
# REDUCED ROUND SPECIFIC ATTACKS
# ═══════════════════════════════════════════════════════════════════════════

class ReducedRoundAttack:
    """Specialized attacks for reduced-round SHA-256"""

    def __init__(self, rounds: int):
        self.rounds = rounds
        self.sha = ReducedSHA256(rounds)
        self.best_collision = None
        self.best_bits = 0

    def linear_cryptanalysis(self, N: int = 100000):
        """Linear approximation attack"""

        print(f"\n   📊 Linear cryptanalysis on {self.rounds}-round SHA-256...")

        # Build linear approximation table
        correlations = defaultdict(list)

        for i in range(1, N + 1):
            h = self.sha.hash_str(i)
            h_int = int(h, 16)

            # Check linear relations between input and output bits
            input_parity = bin(i).count('1') % 2

            for bit_pos in range(256):
                output_bit = (h_int >> bit_pos) & 1
                if output_bit == input_parity:
                    correlations[bit_pos].append(i)

            if i % 25000 == 0:
                print(f"      Analyzed: {i:,}/{N:,}")

        # Find strongest correlations
        strong_positions = sorted(correlations.keys(), key=lambda x: len(correlations[x]), reverse=True)

        print(f"\n   Strongest linear approximations at bit positions: {strong_positions[:5]}")

        # Use strong positions to find collision candidates
        for pos in strong_positions[:10]:
            messages = correlations[pos][:1000]

            for i in range(len(messages)):
                for j in range(i + 1, min(len(messages), i + 100)):
                    m1, m2 = messages[i], messages[j]
                    h1 = self.sha.hash_str(m1)
                    h2 = self.sha.hash_str(m2)
                    bits = count_equal_bits(h1, h2)

                    if bits > self.best_bits:
                        self.best_bits = bits
                        self.best_collision = (m1, m2, h1, h2)
                        print(f"      ✨ Linear: {bits}/256 bits - M1={m1}, M2={m2}")

    def boomerang_attack(self, N: int = 50000):
        """Simplified boomerang attack structure"""

        print(f"\n   🪃 Boomerang attack on {self.rounds}-round SHA-256...")

        # Build forward and backward tables
        forward = {}  # H(m) -> m

        for i in range(1, N + 1):
            h = self.sha.hash_str(i)
            forward[h] = i

            # Check for "related" hashes
            for delta in [1, 2, 4, 8, 16, 32, 64, 128]:
                m_prime = i ^ delta
                if m_prime > 0:
                    h_prime = self.sha.hash_str(m_prime)
                    bits = count_equal_bits(h, h_prime)

                    if bits > self.best_bits:
                        self.best_bits = bits
                        self.best_collision = (i, m_prime, h, h_prime)
                        print(f"      ✨ Boomerang: {bits}/256 bits - Δ={delta}")

            if i % 10000 == 0:
                print(f"      Processed: {i:,}/{N:,}")

    def run_all_attacks(self, N: int = 50000):
        """Run all reduced-round attacks"""

        print("\n" + "="*70)
        print(f"⚔️  REDUCED-ROUND ATTACKS ({self.rounds} rounds)")
        print("="*70)

        self.linear_cryptanalysis(N)
        self.boomerang_attack(N)

        return self.best_collision, self.best_bits

# ═══════════════════════════════════════════════════════════════════════════
# RESULTS DISPLAY
# ═══════════════════════════════════════════════════════════════════════════

def show_results(result: Dict, elapsed: float, rounds: int = 64):
    """Display final results"""

    print("\n" + "="*70)
    print("🏆 FINAL RESULTS")
    print("="*70)

    bits = result['bits']
    pct = bits / 256 * 100

    bar = "█" * int(pct // 5) + "░" * (20 - int(pct // 5))

    rounds_str = f"{rounds} rounds" if rounds < 64 else "FULL (64 rounds)"

    print(f"""
╔═══════════════════════════════════════════════════════════════════════╗
║                                                                       ║
║   SHA-256 VERSION: {rounds_str:<48}║
║                                                                       ║
║   BEST COLLISION: {bits}/256 BITS ({pct:.2f}%)                          ║
║                                                                       ║
║   📝 MESSAGE 1: {str(result['m1']):<54}║
║   📝 MESSAGE 2: {str(result['m2']):<54}║
║                                                                       ║
║   🔐 HASH 1: {result['h1'][:32]}...  ║
║   🔐 HASH 2: {result['h2'][:32]}...  ║
║                                                                       ║
║   📊 [{bar}] {pct:.1f}%                      ║
║   🔧 Method: {str(result['method']):<56}║
║   ⏱️  Time: {elapsed:.1f} seconds                                       ║
║                                                                       ║
╚═══════════════════════════════════════════════════════════════════════╝
    """)

    # Show bit comparison
    if result['h1'] and result['h2']:
        print("\n📊 BIT COMPARISON (first 128 bits):")

        bin1 = bin(int(result['h1'], 16))[2:].zfill(256)
        bin2 = bin(int(result['h2'], 16))[2:].zfill(256)

        print("\n   H1: ", end="")
        for i in range(128):
            if bin1[i] == bin2[i]:
                print(f"\033[92m{bin1[i]}\033[0m", end="")
            else:
                print(f"\033[91m{bin1[i]}\033[0m", end="")
            if (i + 1) % 32 == 0:
                print()
                print("       ", end="")

        print("\n   H2: ", end="")
        for i in range(128):
            if bin1[i] == bin2[i]:
                print(f"\033[92m{bin2[i]}\033[0m", end="")
            else:
                print(f"\033[91m{bin2[i]}\033[0m", end="")
            if (i + 1) % 32 == 0:
                print()
                print("       ", end="")

        print("\n\n   (Green = matching bits, Red = different bits)")

def show_statistics(result: Dict):
    """Show detailed statistics"""

    print("\n" + "="*70)
    print("📈 DETAILED STATISTICS")
    print("="*70)

    bits = result['bits']

    # Expected vs observed
    expected = 128  # Random expectation
    observed = bits
    deviation = observed - expected

    print(f"""
   Expected equal bits (random): 128/256 (50.00%)
   Observed equal bits:          {observed}/256 ({observed/256*100:.2f}%)
   Deviation from random:        +{deviation} bits ({deviation/256*100:.2f}%)

   Improvement factor:           {observed/expected:.2f}x over random
    """)

    # Estimate difficulty
    if bits >= 256:
        print("   🎉 FULL COLLISION FOUND! This should be impossible for SHA-256!")
    elif bits >= 240:
        print("   ⚠️  Near-collision! Extremely significant finding!")
    elif bits >= 200:
        print("   📊 Strong partial collision - worth investigating")
    elif bits >= 180:
        print("   📊 Good partial collision")
    else:
        print("   📊 Partial collision - common in large searches")

# ═══════════════════════════════════════════════════════════════════════════
# MAIN MENU
# ═══════════════════════════════════════════════════════════════════════════

def main_menu():
    """Interactive main menu"""

    show_banner()
    show_theory_extended()

    print("\n" + "="*70)
    print("📋 MAIN MENU")
    print("="*70)

    print("""
   1. Full SHA-256 Analysis (64 rounds) - Educational
   2. Reduced-Round SHA-256 (choose rounds) - More feasible
   3. Quick Demo (fast, fewer samples)
   4. Custom Configuration
   5. Exit
    """)

    try:
        choice = input("   Select option [1-5]: ").strip()
    except:
        choice = "3"

    if choice == "1":
        run_full_sha256()
    elif choice == "2":
        run_reduced_round()
    elif choice == "3":
        run_quick_demo()
    elif choice == "4":
        run_custom()
    else:
        print("\n   Goodbye! 👋")
        return

def run_full_sha256():
    """Run analysis on full SHA-256"""

    print("\n" + "="*70)
    print("🔒 FULL SHA-256 ANALYSIS")
    print("="*70)

    print("""
   ⚠️  IMPORTANT NOTE:

   Full SHA-256 (64 rounds) is cryptographically secure.
   Finding a true collision (256/256 bits) would require
   approximately 2^128 operations - computationally infeasible.

   This analysis will find PARTIAL collisions (>128 bits matching),
   which is educational but doesn't break SHA-256.
    """)

    input("   Press ENTER to continue...")

    try:
        N = int(input("\n   Sample size [50000]: ") or "50000")
        dims = int(input("   Dimensions [8]: ") or "8")
        iters = int(input("   Attractor iterations [10]: ") or "10")
    except:
        N, dims, iters = 50000, 8, 10

    finder = HybridCollisionFinder(dimensions=dims, rounds=64)
    elapsed = finder.run_full_analysis(N=N, iterations=iters)

    show_results(finder.best_result, elapsed, 64)
    show_statistics(finder.best_result)

def run_reduced_round():
    """Run analysis on reduced-round SHA-256"""

    print("\n" + "="*70)
    print("⚡ REDUCED-ROUND SHA-256 ANALYSIS")
    print("="*70)

    print("""
   Available round configurations:

   • 16 rounds - Very weak, collisions more likely
   • 24 rounds - Weak
   • 32 rounds - Moderate
   • 40 rounds - Stronger
   • 48 rounds - Strong (but still weaker than full)

   The fewer rounds, the higher chance of finding strong collisions.
    """)

    try:
        rounds = int(input("\n   Number of rounds [24]: ") or "24")
        N = int(input("   Sample size [100000]: ") or "100000")
    except:
        rounds, N = 24, 100000

    rounds = min(max(rounds, 1), 64)

    # Run hybrid finder with reduced rounds
    finder = HybridCollisionFinder(dimensions=8, rounds=rounds)
    elapsed = finder.run_full_analysis(N=N, iterations=15)

    # Also run specialized reduced-round attacks
    print("\n   Running specialized reduced-round attacks...")
    rr_attack = ReducedRoundAttack(rounds)
    rr_result, rr_bits = rr_attack.run_all_attacks(N // 2)

    # Compare results
    if rr_bits > finder.best_result['bits'] and rr_result is not None:
        finder.best_result = {
            'bits': rr_bits,
            'm1': rr_result[0],
            'm2': rr_result[1],
            'h1': rr_result[2],
            'h2': rr_result[3],
            'method': 'Reduced-Round-Specialized'
        }

    show_results(finder.best_result, elapsed, rounds)
    show_statistics(finder.best_result)

def run_quick_demo():
    """Quick demonstration"""

    print("\n" + "="*70)
    print("🚀 QUICK DEMO")
    print("="*70)

    print("\n   Running quick demo with 16-round SHA-256...")

    finder = HybridCollisionFinder(dimensions=4, rounds=16)

    start = time.time()
    finder.run_nd_analysis(N=20000)
    finder.run_prefix_attack(N=20000)
    finder.run_attractor_analysis(n_attractors=10, iterations=5)
    elapsed = time.time() - start

    show_results(finder.best_result, elapsed, 16)

    print("\n   💡 For better results, try reduced-round analysis with more samples!")

def run_custom():
    """Custom configuration"""

    print("\n" + "="*70)
    print("⚙️  CUSTOM CONFIGURATION")
    print("="*70)

    try:
        rounds = int(input("   SHA-256 rounds [1-64, default=64]: ") or "64")
        dims = int(input("   Embedding dimensions [2-32, default=8]: ") or "8")
        N = int(input("   Sample size [default=50000]: ") or "50000")
        iters = int(input("   Attractor iterations [default=10]: ") or "10")

        print("\n   Transformations to apply:")
        use_poly = input("   Use polynomial? [y/N]: ").lower() == 'y'
        use_trig = input("   Use trigonometric? [y/N]: ").lower() == 'y'
        use_wavelet = input("   Use wavelet? [y/N]: ").lower() == 'y'

    except:
        rounds, dims, N, iters = 64, 8, 50000, 10
        use_poly = use_trig = use_wavelet = False

    finder = HybridCollisionFinder(dimensions=dims, rounds=rounds)
    elapsed = finder.run_full_analysis(N=N, iterations=iters)

    show_results(finder.best_result, elapsed, rounds)
    show_statistics(finder.best_result)

# ═══════════════════════════════════════════════════════════════════════════
# ENTRY POINT
# ═══════════════════════════════════════════════════════════════════════════

if __name__ == "__main__":
    try:
        main_menu()
    except KeyboardInterrupt:
        print("\n\n   Interrupted by user. Goodbye! 👋")
    except Exception as e:
        print(f"\n   Error: {e}")
        import traceback
        traceback.print_exc()

    print("\n" + "="*70)
    print("   Author: Kaoru Aguilera Katayama")
    print("   Extended N-Dimensional Analysis v2.0")
    print("="*70)


╔═══════════════════════════════════════════════════════════════════════════╗
║                                                                           ║
║   ███████╗██╗  ██╗ █████╗       ██████╗ ███████╗ ██████╗                  ║
║   ██╔════╝██║  ██║██╔══██╗      ╚════██╗██╔════╝██╔════╝                  ║
║   ███████╗███████║███████║█████╗ █████╔╝███████╗███████╗                  ║
║   ╚════██║██╔══██║██╔══██║╚════╝██╔═══╝ ╚════██║██╔═══██╗                 ║
║   ███████║██║  ██║██║  ██║      ███████╗███████║╚██████╔╝                 ║
║   ╚══════╝╚═╝  ╚═╝╚═╝  ╚═╝      ╚══════╝╚══════╝ ╚═════╝                  ║
║                                                                           ║
║              COLLISION FINDER v2.0 - EXTENDED                             ║
║                                                                           ║
║   🔷 N-Dimensional Embeddings (4D, 5D, ... nD)                           ║
║   🔷 Non-Linear Dimensional Transformations                     

In [3]:
# ╔═══════════════════════════════════════════════════════════════════════════╗
# ║                                                                           ║
# ║   SHA-256 COLLISION FINDER v3.0 - FULL 64 ROUNDS ONLY                    ║
# ║   N-Dimensional + Non-Linear + Attractors                                 ║
# ║                                                                           ║
# ║   Author: Kaoru Aguilera Katayama                                        ║
# ║   CURRENT RECORD: 174/256 bits (67.97%) - M1=88514, M2=88551             ║
# ║                                                                           ║
# ║   OBJETIVO: SUPERAR 174 BITS → META: 256/256 (COLISIÓN TOTAL)           ║
# ║                                                                           ║
# ╚═══════════════════════════════════════════════════════════════════════════╝

import hashlib
import numpy as np
from collections import defaultdict
from typing import Dict, List, Tuple, Optional, Set
import time
import warnings
import sys
import os

warnings.filterwarnings('ignore')

# ═══════════════════════════════════════════════════════════════════════════
# VERIFIED RECORD
# ═══════════════════════════════════════════════════════════════════════════

VERIFIED_RECORD = {
    'm1': 88514,
    'm2': 88551,
    'bits': 174,
    'percentage': 67.97
}

# ═══════════════════════════════════════════════════════════════════════════
# BANNER
# ═══════════════════════════════════════════════════════════════════════════

def show_banner():
    print("""
╔═══════════════════════════════════════════════════════════════════════════╗
║                                                                           ║
║   ███████╗██╗  ██╗ █████╗       ██████╗ ███████╗ ██████╗                  ║
║   ██╔════╝██║  ██║██╔══██╗      ╚════██╗██╔════╝██╔════╝                  ║
║   ███████╗███████║███████║█████╗ █████╔╝███████╗███████╗                  ║
║   ╚════██║██╔══██║██╔══██║╚════╝██╔═══╝ ╚════██║██╔═══██╗                 ║
║   ███████║██║  ██║██║  ██║      ███████╗███████║╚██████╔╝                 ║
║   ╚══════╝╚═╝  ╚═╝╚═╝  ╚═╝      ╚══════╝╚══════╝ ╚═════╝                  ║
║                                                                           ║
║            FULL 64-ROUND COLLISION HUNTER v3.0                            ║
║                                                                           ║
║   ══════════════════════════════════════════════════════════════════════  ║
║   ║  VERIFIED RECORD: 174/256 bits (67.97%)                            ║  ║
║   ║  M1 = 88514, M2 = 88551                                            ║  ║
║   ║  SHA-256 FULL (64 ROUNDS) - NO SHORTCUTS                           ║  ║
║   ══════════════════════════════════════════════════════════════════════  ║
║                                                                           ║
║   🎯 OBJETIVO: SUPERAR 174 BITS                                          ║
║   🏆 META FINAL: 256/256 BITS (COLISIÓN TOTAL)                           ║
║                                                                           ║
║   Author: Kaoru Aguilera Katayama                                        ║
║                                                                           ║
╚═══════════════════════════════════════════════════════════════════════════╝
    """)

def verify_record():
    """Verify the claimed record"""
    print("\n" + "="*70)
    print("✅ VERIFICANDO RÉCORD ACTUAL")
    print("="*70)

    m1, m2 = 88514, 88551
    h1 = hashlib.sha256(str(m1).encode()).hexdigest()
    h2 = hashlib.sha256(str(m2).encode()).hexdigest()

    b1 = bin(int(h1, 16))[2:].zfill(256)
    b2 = bin(int(h2, 16))[2:].zfill(256)

    matching = sum(x == y for x, y in zip(b1, b2))

    print(f"""
   M1 = {m1}
   M2 = {m2}

   SHA-256(M1) = {h1}
   SHA-256(M2) = {h2}

   Bits iguales: {matching}/256 ({100*matching/256:.2f}%)

   ✅ RÉCORD VERIFICADO: {matching} bits coincidentes
   """)

    return matching

# ═══════════════════════════════════════════════════════════════════════════
# CORE FUNCTIONS - SHA-256 FULL 64 ROUNDS
# ═══════════════════════════════════════════════════════════════════════════

def sha256(msg) -> str:
    """SHA-256 COMPLETO - 64 rondas, sin atajos"""
    return hashlib.sha256(str(msg).encode()).hexdigest()

def sha256_int(msg) -> int:
    """SHA-256 como entero de 256 bits"""
    return int(hashlib.sha256(str(msg).encode()).hexdigest(), 16)

def sha256_bits(msg) -> str:
    """SHA-256 como string binario"""
    return bin(sha256_int(msg))[2:].zfill(256)

def count_equal_bits(h1: str, h2: str) -> int:
    """Contar bits idénticos entre dos hashes"""
    b1 = bin(int(h1, 16))[2:].zfill(256)
    b2 = bin(int(h2, 16))[2:].zfill(256)
    return sum(a == b for a, b in zip(b1, b2))

def hamming_distance(h1: str, h2: str) -> int:
    """Distancia Hamming entre hashes"""
    return 256 - count_equal_bits(h1, h2)

def xor_hashes(h1: str, h2: str) -> str:
    """XOR de dos hashes"""
    return format(int(h1, 16) ^ int(h2, 16), '064x')

def hash_to_vector(h: str, dimensions: int) -> np.ndarray:
    """Convertir hash a vector n-dimensional"""
    bits_per_dim = 256 // dimensions
    h_int = int(h, 16)
    vector = []

    for i in range(dimensions):
        chunk = (h_int >> (i * bits_per_dim)) & ((1 << bits_per_dim) - 1)
        normalized = chunk / (2 ** bits_per_dim)
        vector.append(normalized)

    return np.array(vector)

# ═══════════════════════════════════════════════════════════════════════════
# RECORD TRACKER
# ═══════════════════════════════════════════════════════════════════════════

class RecordTracker:
    """Rastrea el mejor resultado encontrado"""

    def __init__(self):
        self.best_bits = 174  # Tu récord actual
        self.best_m1 = 88514
        self.best_m2 = 88551
        self.best_h1 = sha256(88514)
        self.best_h2 = sha256(88551)
        self.history = []
        self.improvements = 0

    def check_and_update(self, m1: int, m2: int, h1: str = None, h2: str = None) -> bool:
        """Verifica y actualiza si hay mejora"""
        if m1 == m2:
            return False

        if h1 is None:
            h1 = sha256(m1)
        if h2 is None:
            h2 = sha256(m2)

        bits = count_equal_bits(h1, h2)

        if bits > self.best_bits:
            self.best_bits = bits
            self.best_m1 = m1
            self.best_m2 = m2
            self.best_h1 = h1
            self.best_h2 = h2
            self.improvements += 1

            self.history.append({
                'bits': bits,
                'm1': m1,
                'm2': m2,
                'time': time.time()
            })

            return True
        return False

    def print_status(self, prefix: str = ""):
        """Imprime estado actual"""
        pct = self.best_bits / 256 * 100
        print(f"   {prefix}🏆 MEJOR: {self.best_bits}/256 ({pct:.2f}%) - M1={self.best_m1}, M2={self.best_m2}")

# ═══════════════════════════════════════════════════════════════════════════
# N-DIMENSIONAL SPACE ANALYSIS
# ═══════════════════════════════════════════════════════════════════════════

class NDimensionalCollisionFinder:
    """Búsqueda en espacio N-dimensional"""

    def __init__(self, tracker: RecordTracker, dimensions: int = 8):
        self.tracker = tracker
        self.dimensions = dimensions
        self.points = []
        self.hashes = []
        self.messages = []

    def build_space(self, N: int):
        """Construir espacio N-dimensional"""

        print(f"\n   📐 Construyendo espacio {self.dimensions}D con {N:,} puntos...")

        self.points = []
        self.hashes = []
        self.messages = []

        for i in range(1, N + 1):
            h = sha256(i)
            vec = hash_to_vector(h, self.dimensions)

            self.points.append(vec)
            self.hashes.append(h)
            self.messages.append(i)

            if i % 50000 == 0:
                print(f"      Procesados: {i:,}/{N:,}")

        self.points = np.array(self.points)
        print(f"   ✓ Espacio construido: {self.points.shape}")

    def find_collisions_bruteforce(self, sample_size: int = 10000):
        """Búsqueda por fuerza bruta en espacio dimensional"""

        print(f"\n   🔍 Buscando en espacio {self.dimensions}D...")

        n = len(self.points)
        checked = 0

        # Calcular distancias y ordenar
        for i in range(min(n, sample_size)):
            point_i = self.points[i]

            # Calcular distancia a todos los demás
            distances = np.linalg.norm(self.points - point_i, axis=1)

            # Obtener los más cercanos
            nearest_idx = np.argsort(distances)[1:21]  # Top 20 más cercanos

            for j in nearest_idx:
                if i != j:
                    m1, m2 = self.messages[i], self.messages[j]
                    h1, h2 = self.hashes[i], self.hashes[j]

                    if self.tracker.check_and_update(m1, m2, h1, h2):
                        print(f"      ✨ ¡NUEVO RÉCORD! {self.tracker.best_bits}/256 bits")
                        self.tracker.print_status("      ")

            checked += 1
            if checked % 2000 == 0:
                print(f"      Verificados: {checked:,}/{sample_size:,}")

    def find_collisions_kdtree(self, k: int = 50):
        """Búsqueda usando KD-Tree si scipy está disponible"""

        try:
            from scipy.spatial import KDTree

            print(f"\n   🌳 Usando KD-Tree para búsqueda eficiente...")

            tree = KDTree(self.points)

            checked = 0
            for i, point in enumerate(self.points):
                distances, indices = tree.query(point, k=k)

                for j, idx in enumerate(indices[1:]):  # Skip self
                    if idx != i and idx < len(self.messages):
                        m1, m2 = self.messages[i], self.messages[idx]
                        h1, h2 = self.hashes[i], self.hashes[idx]

                        if self.tracker.check_and_update(m1, m2, h1, h2):
                            print(f"      ✨ ¡NUEVO RÉCORD! {self.tracker.best_bits}/256 bits")

                checked += 1
                if checked % 10000 == 0:
                    print(f"      Procesados: {checked:,}/{len(self.points):,}")

        except ImportError:
            print("   ⚠️ scipy no disponible, usando método alternativo...")
            self.find_collisions_bruteforce()

# ═══════════════════════════════════════════════════════════════════════════
# NON-LINEAR TRANSFORMATIONS
# ═══════════════════════════════════════════════════════════════════════════

class NonLinearAnalyzer:
    """Análisis con transformaciones no lineales"""

    def __init__(self, tracker: RecordTracker):
        self.tracker = tracker

    def analyze_polynomial_space(self, N: int = 50000, degree: int = 3):
        """Espacio polinomial"""

        print(f"\n   📈 Análisis en espacio polinomial (grado {degree})...")

        # Construir representaciones polinomiales
        poly_vectors = []
        hashes = []
        messages = []

        for i in range(1, N + 1):
            h = sha256(i)
            h_int = int(h, 16)

            # Crear características polinomiales de fragmentos del hash
            features = []
            for chunk_start in range(0, 256, 32):
                chunk = (h_int >> chunk_start) & 0xFFFFFFFF
                normalized = chunk / (2**32)

                # Términos polinomiales
                for d in range(1, degree + 1):
                    features.append(normalized ** d)

            poly_vectors.append(features)
            hashes.append(h)
            messages.append(i)

            if i % 25000 == 0:
                print(f"      Procesados: {i:,}/{N:,}")

        poly_vectors = np.array(poly_vectors)

        # Buscar vecinos cercanos
        print(f"      Buscando en espacio de {poly_vectors.shape[1]} dimensiones...")

        sample_indices = np.random.choice(len(poly_vectors), min(5000, len(poly_vectors)), replace=False)

        for idx in sample_indices:
            vec = poly_vectors[idx]
            distances = np.linalg.norm(poly_vectors - vec, axis=1)
            nearest = np.argsort(distances)[1:11]

            for j in nearest:
                m1, m2 = messages[idx], messages[j]
                h1, h2 = hashes[idx], hashes[j]

                if self.tracker.check_and_update(m1, m2, h1, h2):
                    print(f"      ✨ ¡POLYNOMIAL RÉCORD! {self.tracker.best_bits}/256 bits")

    def analyze_trigonometric_space(self, N: int = 50000):
        """Espacio trigonométrico"""

        print(f"\n   🌊 Análisis en espacio trigonométrico...")

        trig_vectors = []
        hashes = []
        messages = []

        frequencies = [1, 2, 4, 8, 16]

        for i in range(1, N + 1):
            h = sha256(i)
            h_int = int(h, 16)

            features = []
            for chunk_start in range(0, 256, 64):
                chunk = (h_int >> chunk_start) & ((1 << 64) - 1)
                normalized = chunk / (2**64)

                for freq in frequencies:
                    features.append(np.sin(2 * np.pi * freq * normalized))
                    features.append(np.cos(2 * np.pi * freq * normalized))

            trig_vectors.append(features)
            hashes.append(h)
            messages.append(i)

            if i % 25000 == 0:
                print(f"      Procesados: {i:,}/{N:,}")

        trig_vectors = np.array(trig_vectors)

        # Buscar
        print(f"      Dimensiones: {trig_vectors.shape[1]}")

        sample_indices = np.random.choice(len(trig_vectors), min(5000, len(trig_vectors)), replace=False)

        for idx in sample_indices:
            vec = trig_vectors[idx]
            distances = np.linalg.norm(trig_vectors - vec, axis=1)
            nearest = np.argsort(distances)[1:11]

            for j in nearest:
                m1, m2 = messages[idx], messages[j]
                h1, h2 = hashes[idx], hashes[j]

                if self.tracker.check_and_update(m1, m2, h1, h2):
                    print(f"      ✨ ¡TRIG RÉCORD! {self.tracker.best_bits}/256 bits")

    def analyze_wavelet_space(self, N: int = 50000):
        """Análisis tipo wavelet"""

        print(f"\n   〰️ Análisis wavelet...")

        scales = [1, 2, 4, 8, 16, 32]

        wavelet_vectors = []
        hashes = []
        messages = []

        for i in range(1, N + 1):
            h = sha256(i)
            bits = sha256_bits(i)

            # Convertir bits a señal
            signal = np.array([int(b) for b in bits], dtype=float)

            features = []
            for scale in scales:
                # Promedios en diferentes escalas
                for start in range(0, 256, scale):
                    end = min(start + scale, 256)
                    avg = np.mean(signal[start:end])
                    features.append(avg)

            wavelet_vectors.append(features)
            hashes.append(h)
            messages.append(i)

            if i % 25000 == 0:
                print(f"      Procesados: {i:,}/{N:,}")

        wavelet_vectors = np.array(wavelet_vectors)

        # Buscar
        sample_indices = np.random.choice(len(wavelet_vectors), min(5000, len(wavelet_vectors)), replace=False)

        for idx in sample_indices:
            vec = wavelet_vectors[idx]
            distances = np.linalg.norm(wavelet_vectors - vec, axis=1)
            nearest = np.argsort(distances)[1:11]

            for j in nearest:
                m1, m2 = messages[idx], messages[j]
                h1, h2 = hashes[idx], hashes[j]

                if self.tracker.check_and_update(m1, m2, h1, h2):
                    print(f"      ✨ ¡WAVELET RÉCORD! {self.tracker.best_bits}/256 bits")

# ═══════════════════════════════════════════════════════════════════════════
# ATTRACTOR REFINEMENT
# ═══════════════════════════════════════════════════════════════════════════

class AttractorSystem:
    """Sistema de atractores iterativos"""

    def __init__(self, tracker: RecordTracker):
        self.tracker = tracker
        self.attractors = []

    def initialize_from_record(self):
        """Inicializar atractores desde el récord conocido"""

        # Atractor 1: Alrededor del récord actual
        self.attractors.append({
            'center': (self.tracker.best_m1 + self.tracker.best_m2) // 2,
            'radius': 1000,
            'best_bits': self.tracker.best_bits
        })

        # Atractores adicionales en regiones prometedoras
        promising_regions = [
            (88500, 500),    # Cerca del récord
            (100000, 1000),
            (50000, 1000),
            (150000, 1000),
            (200000, 1000),
            (1000, 500),
            (10000, 500),
        ]

        for center, radius in promising_regions:
            self.attractors.append({
                'center': center,
                'radius': radius,
                'best_bits': 0
            })

    def evaluate_region(self, center: int, radius: int, samples: int = 2000) -> Tuple[int, int, int]:
        """Evaluar una región y encontrar el mejor par"""

        best_pair = None
        best_bits = 0

        # Generar puntos en la región
        points = set()
        points.add(center)

        for _ in range(samples):
            offset = np.random.randint(-radius, radius + 1)
            point = max(1, center + offset)
            points.add(point)

        points = list(points)

        # Pre-calcular hashes
        hash_cache = {p: sha256(p) for p in points}

        # Buscar mejor par
        for i in range(len(points)):
            for j in range(i + 1, len(points)):
                p1, p2 = points[i], points[j]
                h1, h2 = hash_cache[p1], hash_cache[p2]

                bits = count_equal_bits(h1, h2)

                if bits > best_bits:
                    best_bits = bits
                    best_pair = (p1, p2)

                if self.tracker.check_and_update(p1, p2, h1, h2):
                    pass  # Actualizado internamente

        return best_pair[0] if best_pair else center, best_pair[1] if best_pair else center, best_bits

    def run_iterations(self, iterations: int = 20, samples_per_iter: int = 1500):
        """Ejecutar iteraciones de refinamiento"""

        print(f"\n   🎯 Ejecutando {iterations} iteraciones de atractores...")
        print(f"      {len(self.attractors)} atractores iniciales")

        for iter_num in range(iterations):
            print(f"\n      Iteración {iter_num + 1}/{iterations}:")

            for i, att in enumerate(self.attractors):
                m1, m2, bits = self.evaluate_region(
                    att['center'],
                    att['radius'],
                    samples_per_iter
                )

                if bits > att['best_bits']:
                    att['best_bits'] = bits
                    # Mover centro hacia el mejor par
                    att['center'] = (m1 + m2) // 2

                    if bits > 174:  # Supera récord original
                        print(f"         🔥 Atractor {i}: {bits} bits - ¡Supera 174!")

                # Reducir radio gradualmente
                att['radius'] = max(100, int(att['radius'] * 0.85))

            # Status
            self.tracker.print_status(f"      ")

    def intensive_local_search(self, m1: int, m2: int, radius: int = 50, iterations: int = 100):
        """Búsqueda intensiva local alrededor de un par prometedor"""

        print(f"\n   🔬 Búsqueda intensiva alrededor de ({m1}, {m2})...")

        current_best = count_equal_bits(sha256(m1), sha256(m2))

        for iter_num in range(iterations):
            improved = False

            for d1 in range(-radius, radius + 1, max(1, radius // 20)):
                for d2 in range(-radius, radius + 1, max(1, radius // 20)):
                    new_m1 = max(1, m1 + d1)
                    new_m2 = max(1, m2 + d2)

                    if new_m1 != new_m2:
                        if self.tracker.check_and_update(new_m1, new_m2):
                            if self.tracker.best_bits > current_best:
                                current_best = self.tracker.best_bits
                                m1, m2 = self.tracker.best_m1, self.tracker.best_m2
                                improved = True
                                print(f"         Iter {iter_num}: {current_best}/256 bits")

            if not improved:
                radius = max(10, radius // 2)

            if radius < 10:
                break

# ═══════════════════════════════════════════════════════════════════════════
# PREFIX COLLISION ATTACK
# ═══════════════════════════════════════════════════════════════════════════

class PrefixAttack:
    """Ataque basado en prefijos de hash"""

    def __init__(self, tracker: RecordTracker):
        self.tracker = tracker

    def run(self, N: int = 200000):
        """Ejecutar ataque de prefijos"""

        print(f"\n   🔤 Ataque de prefijos con {N:,} mensajes...")

        prefixes = defaultdict(list)

        for i in range(1, N + 1):
            h = sha256(i)

            # Guardar por múltiples longitudes de prefijo
            for length in [5, 6, 7, 8, 9, 10]:
                prefixes[(length, h[:length])].append((i, h))

            if i % 50000 == 0:
                print(f"      Indexados: {i:,}/{N:,}")

        # Buscar colisiones
        for length in [10, 9, 8, 7, 6, 5]:
            groups = [(k, v) for k, v in prefixes.items() if k[0] == length and len(v) >= 2]

            if groups:
                print(f"      Prefijo-{length}: {len(groups)} grupos con 2+ elementos")

            for (_, prefix), group in groups:
                for i in range(len(group)):
                    for j in range(i + 1, len(group)):
                        m1, h1 = group[i]
                        m2, h2 = group[j]

                        if m1 != m2:
                            if self.tracker.check_and_update(m1, m2, h1, h2):
                                print(f"         ✨ ¡PREFIX-{length} RÉCORD! {self.tracker.best_bits}/256")

# ═══════════════════════════════════════════════════════════════════════════
# DIFFERENTIAL ANALYSIS
# ═══════════════════════════════════════════════════════════════════════════

class DifferentialAnalyzer:
    """Análisis diferencial"""

    def __init__(self, tracker: RecordTracker):
        self.tracker = tracker

    def xor_differential(self, N: int = 100000):
        """Análisis con diferencias XOR"""

        print(f"\n   ⊕ Análisis diferencial XOR...")

        # Deltas a probar
        deltas = [1, 2, 3, 4, 5, 7, 8, 15, 16, 31, 32, 37, 63, 64, 127, 128, 255, 256]

        for delta in deltas:
            found_this_delta = 0

            for m in range(1, N + 1):
                m2 = m ^ delta

                if m2 > 0 and m != m2:
                    if self.tracker.check_and_update(m, m2):
                        found_this_delta += 1
                        if self.tracker.best_bits > 174:
                            print(f"         ✨ XOR Δ={delta}: {self.tracker.best_bits}/256 bits")

            if found_this_delta > 0:
                print(f"      Δ={delta}: {found_this_delta} mejoras encontradas")

    def additive_differential(self, N: int = 100000):
        """Análisis con diferencias aditivas"""

        print(f"\n   ➕ Análisis diferencial aditivo...")

        deltas = [1, 2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]

        for delta in deltas:
            for m in range(1, N + 1):
                m2 = m + delta

                if self.tracker.check_and_update(m, m2):
                    if self.tracker.best_bits > 174:
                        print(f"         ✨ ADD Δ={delta}: {self.tracker.best_bits}/256 bits")

    def neighborhood_search(self, center: int, radius: int = 100):
        """Búsqueda exhaustiva en vecindario"""

        print(f"\n   🔍 Búsqueda en vecindario de {center}±{radius}...")

        points = range(max(1, center - radius), center + radius + 1)
        hashes = {p: sha256(p) for p in points}

        for p1 in points:
            for p2 in points:
                if p1 < p2:
                    if self.tracker.check_and_update(p1, p2, hashes[p1], hashes[p2]):
                        if self.tracker.best_bits > 174:
                            print(f"         ✨ NEIGHBOR: {self.tracker.best_bits}/256 bits")

# ═══════════════════════════════════════════════════════════════════════════
# BIT PATTERN ANALYSIS
# ═══════════════════════════════════════════════════════════════════════════

class BitPatternAnalyzer:
    """Análisis de patrones de bits"""

    def __init__(self, tracker: RecordTracker):
        self.tracker = tracker

    def analyze_bit_correlations(self, N: int = 50000):
        """Analizar correlaciones entre bits de entrada y salida"""

        print(f"\n   🧬 Analizando correlaciones de bits...")

        # Agrupar por número de bits en 1 en la entrada
        by_popcount = defaultdict(list)

        for i in range(1, N + 1):
            popcount = bin(i).count('1')
            h = sha256(i)
            by_popcount[popcount].append((i, h))

            if i % 25000 == 0:
                print(f"      Procesados: {i:,}/{N:,}")

        # Buscar colisiones dentro de grupos
        for popcount, group in by_popcount.items():
            if len(group) >= 2:
                for i in range(min(len(group), 500)):
                    for j in range(i + 1, min(len(group), 500)):
                        m1, h1 = group[i]
                        m2, h2 = group[j]

                        if self.tracker.check_and_update(m1, m2, h1, h2):
                            if self.tracker.best_bits > 174:
                                print(f"         ✨ POPCOUNT-{popcount}: {self.tracker.best_bits}/256")

    def analyze_run_lengths(self, N: int = 50000):
        """Analizar longitudes de secuencias"""

        print(f"\n   📏 Analizando longitudes de secuencias...")

        def get_run_signature(bits: str) -> Tuple:
            """Obtener firma de longitudes de secuencias"""
            runs = []
            current_run = 1
            for i in range(1, len(bits)):
                if bits[i] == bits[i-1]:
                    current_run += 1
                else:
                    runs.append(current_run)
                    current_run = 1
            runs.append(current_run)
            return tuple(runs[:10])  # Primeras 10 secuencias

        by_signature = defaultdict(list)

        for i in range(1, N + 1):
            h = sha256(i)
            bits = bin(int(h, 16))[2:].zfill(256)
            sig = get_run_signature(bits)
            by_signature[sig].append((i, h))

            if i % 25000 == 0:
                print(f"      Procesados: {i:,}/{N:,}")

        # Buscar en grupos con misma firma
        for sig, group in by_signature.items():
            if len(group) >= 2:
                for i in range(min(len(group), 100)):
                    for j in range(i + 1, min(len(group), 100)):
                        m1, h1 = group[i]
                        m2, h2 = group[j]

                        if self.tracker.check_and_update(m1, m2, h1, h2):
                            if self.tracker.best_bits > 174:
                                print(f"         ✨ RUN-SIG: {self.tracker.best_bits}/256")

# ═══════════════════════════════════════════════════════════════════════════
# GENETIC/EVOLUTIONARY SEARCH
# ═══════════════════════════════════════════════════════════════════════════

class GeneticCollisionFinder:
    """Búsqueda evolutiva de colisiones"""

    def __init__(self, tracker: RecordTracker, population_size: int = 100):
        self.tracker = tracker
        self.population_size = population_size
        self.population = []

    def initialize_population(self):
        """Inicializar población"""

        self.population = []

        # Incluir el récord conocido
        self.population.append((self.tracker.best_m1, self.tracker.best_m2))

        # Generar población adicional
        while len(self.population) < self.population_size:
            m1 = np.random.randint(1, 500000)
            m2 = np.random.randint(1, 500000)
            if m1 != m2:
                self.population.append((m1, m2))

    def fitness(self, pair: Tuple[int, int]) -> int:
        """Calcular fitness (bits iguales)"""
        m1, m2 = pair
        return count_equal_bits(sha256(m1), sha256(m2))

    def mutate(self, pair: Tuple[int, int]) -> Tuple[int, int]:
        """Mutar un par"""
        m1, m2 = pair

        mutation_type = np.random.choice(['small', 'medium', 'large', 'xor'])

        if mutation_type == 'small':
            m1 = max(1, m1 + np.random.randint(-10, 11))
            m2 = max(1, m2 + np.random.randint(-10, 11))
        elif mutation_type == 'medium':
            m1 = max(1, m1 + np.random.randint(-100, 101))
            m2 = max(1, m2 + np.random.randint(-100, 101))
        elif mutation_type == 'large':
            m1 = max(1, m1 + np.random.randint(-1000, 1001))
            m2 = max(1, m2 + np.random.randint(-1000, 1001))
        else:  # xor
            m1 = max(1, m1 ^ np.random.randint(1, 256))
            m2 = max(1, m2 ^ np.random.randint(1, 256))

        if m1 == m2:
            m2 = m1 + 1

        return (m1, m2)

    def crossover(self, pair1: Tuple[int, int], pair2: Tuple[int, int]) -> Tuple[int, int]:
        """Cruzar dos pares"""
        if np.random.random() < 0.5:
            return (pair1[0], pair2[1])
        else:
            return (pair2[0], pair1[1])

    def evolve(self, generations: int = 100):
        """Ejecutar evolución"""

        print(f"\n   🧬 Evolución genética: {generations} generaciones...")

        self.initialize_population()

        for gen in range(generations):
            # Evaluar fitness
            fitness_scores = [(pair, self.fitness(pair)) for pair in self.population]
            fitness_scores.sort(key=lambda x: x[1], reverse=True)

            # Actualizar tracker con el mejor
            best_pair, best_fitness = fitness_scores[0]
            self.tracker.check_and_update(best_pair[0], best_pair[1])

            if gen % 20 == 0:
                print(f"      Gen {gen}: Mejor fitness = {best_fitness}/256")

            # Selección
            survivors = [pair for pair, _ in fitness_scores[:self.population_size // 2]]

            # Nueva generación
            new_population = survivors.copy()

            while len(new_population) < self.population_size:
                if np.random.random() < 0.7:
                    # Mutación
                    parent = survivors[np.random.randint(len(survivors))]
                    child = self.mutate(parent)
                else:
                    # Crossover
                    p1 = survivors[np.random.randint(len(survivors))]
                    p2 = survivors[np.random.randint(len(survivors))]
                    child = self.crossover(p1, p2)

                new_population.append(child)

            self.population = new_population

        self.tracker.print_status("      Final: ")

# ═══════════════════════════════════════════════════════════════════════════
# MAIN COLLISION HUNTER
# ═══════════════════════════════════════════════════════════════════════════

class SHA256CollisionHunter:
    """Cazador principal de colisiones SHA-256"""

    def __init__(self):
        self.tracker = RecordTracker()
        self.start_time = None

    def run_all_methods(self, N: int = 100000, intensive: bool = True):
        """Ejecutar todos los métodos de búsqueda"""

        self.start_time = time.time()

        print("\n" + "="*70)
        print("🚀 INICIANDO CACERÍA DE COLISIONES SHA-256 (64 RONDAS)")
        print("="*70)
        print(f"\n   Tamaño de muestra: {N:,}")
        print(f"   Modo intensivo: {'SÍ' if intensive else 'NO'}")
        print(f"   Récord a superar: 174/256 bits (67.97%)")

        # 1. Análisis N-dimensional
        print("\n" + "="*70)
        print("📐 FASE 1: ANÁLISIS N-DIMENSIONAL")
        print("="*70)

        for dims in [4, 8, 16, 32]:
            nd_finder = NDimensionalCollisionFinder(self.tracker, dimensions=dims)
            nd_finder.build_space(N)
            nd_finder.find_collisions_kdtree(k=30)

        # 2. Transformaciones no lineales
        print("\n" + "="*70)
        print("🔄 FASE 2: TRANSFORMACIONES NO LINEALES")
        print("="*70)

        nl_analyzer = NonLinearAnalyzer(self.tracker)
        nl_analyzer.analyze_polynomial_space(N // 2, degree=3)
        nl_analyzer.analyze_trigonometric_space(N // 2)
        nl_analyzer.analyze_wavelet_space(N // 2)

        # 3. Sistema de atractores
        print("\n" + "="*70)
        print("🎯 FASE 3: REFINAMIENTO DE ATRACTORES")
        print("="*70)

        attractor = AttractorSystem(self.tracker)
        attractor.initialize_from_record()
        attractor.run_iterations(iterations=25, samples_per_iter=2000)

        # 4. Ataque de prefijos
        print("\n" + "="*70)
        print("🔤 FASE 4: ATAQUE DE PREFIJOS")
        print("="*70)

        prefix = PrefixAttack(self.tracker)
        prefix.run(N * 2)

        # 5. Análisis diferencial
        print("\n" + "="*70)
        print("⊕ FASE 5: ANÁLISIS DIFERENCIAL")
        print("="*70)

        diff = DifferentialAnalyzer(self.tracker)
        diff.xor_differential(N)
        diff.additive_differential(N)

        # 6. Patrones de bits
        print("\n" + "="*70)
        print("🧬 FASE 6: ANÁLISIS DE PATRONES")
        print("="*70)

        pattern = BitPatternAnalyzer(self.tracker)
        pattern.analyze_bit_correlations(N)
        pattern.analyze_run_lengths(N)

        # 7. Búsqueda genética
        print("\n" + "="*70)
        print("🧬 FASE 7: BÚSQUEDA EVOLUTIVA")
        print("="*70)

        genetic = GeneticCollisionFinder(self.tracker, population_size=200)
        genetic.evolve(generations=150)

        # 8. Refinamiento intensivo final
        if intensive:
            print("\n" + "="*70)
            print("🔬 FASE 8: REFINAMIENTO INTENSIVO")
            print("="*70)

            attractor.intensive_local_search(
                self.tracker.best_m1,
                self.tracker.best_m2,
                radius=100,
                iterations=200
            )

            # Búsqueda en vecindario del mejor
            diff.neighborhood_search(self.tracker.best_m1, radius=200)
            diff.neighborhood_search(self.tracker.best_m2, radius=200)

        elapsed = time.time() - self.start_time
        return elapsed

    def show_final_results(self, elapsed: float):
        """Mostrar resultados finales"""

        print("\n" + "="*70)
        print("🏆 RESULTADOS FINALES")
        print("="*70)

        bits = self.tracker.best_bits
        pct = bits / 256 * 100
        m1 = self.tracker.best_m1
        m2 = self.tracker.best_m2
        h1 = self.tracker.best_h1
        h2 = self.tracker.best_h2

        bar = "█" * int(pct // 5) + "░" * (20 - int(pct // 5))

        beat_record = "🎉 ¡NUEVO RÉCORD!" if bits > 174 else "📊 Récord mantenido"

        print(f"""
╔═══════════════════════════════════════════════════════════════════════════╗
║                                                                           ║
║   {beat_record:^67} ║
║                                                                           ║
║   BITS COINCIDENTES: {bits}/256 ({pct:.2f}%)                               ║
║   BARRA: [{bar}]                              ║
║                                                                           ║
║   M1 = {m1:<65}║
║   M2 = {m2:<65}║
║                                                                           ║
║   SHA-256(M1) = {h1}║
║   SHA-256(M2) = {h2}║
║                                                                           ║
║   ⏱️  Tiempo total: {elapsed:.1f} segundos                                  ║
║   📈 Mejoras encontradas: {self.tracker.improvements:<42}║
║                                                                           ║
╚═══════════════════════════════════════════════════════════════════════════╝
        """)

        # Mapa de bits
        print("\n📊 MAPA DE BITS (█=igual, ░=diferente):")

        b1 = bin(int(h1, 16))[2:].zfill(256)
        b2 = bin(int(h2, 16))[2:].zfill(256)

        for row in range(4):
            line = "   "
            for col in range(64):
                idx = row * 64 + col
                if b1[idx] == b2[idx]:
                    line += "█"
                else:
                    line += "░"
            print(line)

        # Estadísticas
        print(f"""
   ═══════════════════════════════════════════════════════════

   📊 ESTADÍSTICAS:

   • Bits esperados (aleatorio): 128/256 (50.00%)
   • Bits obtenidos: {bits}/256 ({pct:.2f}%)
   • Desviación del azar: +{bits - 128} bits (+{(bits-128)/256*100:.2f}%)
   • Factor de mejora: {bits/128:.2f}x sobre aleatorio

   ═══════════════════════════════════════════════════════════
        """)

        # Verificación
        print("   ✅ VERIFICACIÓN:")
        print(f"      M1 ≠ M2: {'SÍ' if m1 != m2 else 'NO'}")
        print(f"      SHA-256 completo (64 rondas): SÍ")

        verify_h1 = hashlib.sha256(str(m1).encode()).hexdigest()
        verify_h2 = hashlib.sha256(str(m2).encode()).hexdigest()
        verify_bits = count_equal_bits(verify_h1, verify_h2)

        print(f"      Verificación independiente: {verify_bits}/256 bits ({'✅ CORRECTO' if verify_bits == bits else '❌ ERROR'})")

# ═══════════════════════════════════════════════════════════════════════════
# MAIN
# ═══════════════════════════════════════════════════════════════════════════

def main():
    show_banner()

    # Verificar récord actual
    verified_bits = verify_record()

    print("\n" + "="*70)
    print("⚙️  CONFIGURACIÓN")
    print("="*70)

    print("""
   Opciones de búsqueda:

   1. Rápida     (N=50,000)   - ~5 minutos
   2. Normal     (N=100,000)  - ~15 minutos
   3. Intensiva  (N=200,000)  - ~30 minutos
   4. Extrema    (N=500,000)  - ~1+ hora
   5. Personalizada
    """)

    try:
        choice = input("   Selecciona [1-5, default=2]: ").strip() or "2"
    except:
        choice = "2"

    configs = {
        "1": (50000, False),
        "2": (100000, True),
        "3": (200000, True),
        "4": (500000, True),
    }

    if choice in configs:
        N, intensive = configs[choice]
    else:
        try:
            N = int(input("   Tamaño de muestra [100000]: ") or "100000")
            intensive = input("   Modo intensivo? [S/n]: ").lower() != 'n'
        except:
            N, intensive = 100000, True

    input(f"\n   Presiona ENTER para comenzar la búsqueda (N={N:,})...")

    # Ejecutar búsqueda
    hunter = SHA256CollisionHunter()
    elapsed = hunter.run_all_methods(N=N, intensive=intensive)

    # Mostrar resultados
    hunter.show_final_results(elapsed)

    print("\n" + "="*70)
    print("   ✅ ANÁLISIS COMPLETADO")
    print("   Author: Kaoru Aguilera Katayama")
    print("   SHA-256 Full 64-Round Collision Hunter v3.0")
    print("="*70)

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n\n   Interrumpido. ¡Hasta pronto! 👋")
    except Exception as e:
        print(f"\n   Error: {e}")
        import traceback
        traceback.print_exc()


╔═══════════════════════════════════════════════════════════════════════════╗
║                                                                           ║
║   ███████╗██╗  ██╗ █████╗       ██████╗ ███████╗ ██████╗                  ║
║   ██╔════╝██║  ██║██╔══██╗      ╚════██╗██╔════╝██╔════╝                  ║
║   ███████╗███████║███████║█████╗ █████╔╝███████╗███████╗                  ║
║   ╚════██║██╔══██║██╔══██║╚════╝██╔═══╝ ╚════██║██╔═══██╗                 ║
║   ███████║██║  ██║██║  ██║      ███████╗███████║╚██████╔╝                 ║
║   ╚══════╝╚═╝  ╚═╝╚═╝  ╚═╝      ╚══════╝╚══════╝ ╚═════╝                  ║
║                                                                           ║
║            FULL 64-ROUND COLLISION HUNTER v3.0                            ║
║                                                                           ║
║   ══════════════════════════════════════════════════════════════════════  ║
║   ║  VERIFIED RECORD: 174/256 bits (67.97%)                  